# Use Requests to fetch jokes from the Chuck Norris API

This notebook demonstrates how to use the `requests` library to interact with a public REST API.
We'll hit four endpoints on [api.chucknorris.io](https://api.chucknorris.io):

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/jokes/random` | GET | Fetch a single random joke |
| `/jokes/categories` | GET | List all available categories |
| `/jokes/random?category={cat}` | GET | Random joke from a specific category |
| `/jokes/search?query={text}` | GET | Search jokes by keyword (min 3 chars) |

## Imports

We need `requests` to make HTTP calls and `json` to pretty-print raw API responses.

In [ ]:
import requests
import json

## 1. Fetch a random Chuck Norris joke

**Endpoint:** `GET https://api.chucknorris.io/jokes/random`

Returns a single joke object with keys like `value` (the joke text), `id`, `url`, and `categories`.
The function below wraps the request in a try/except so network errors are handled gracefully.

In [ ]:
def get_random_joke():
    """
    Fetch a random Chuck Norris joke.

    Returns:
        Dictionary containing joke data, or None if request fails
    """
    url = "https://api.chucknorris.io/jokes/random"

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching joke: {e}")
        return None

Run the cell below to get a random joke. After the joke prints, you'll be asked whether
you want to see the full JSON response from the API — useful for understanding the data structure.

In [ ]:
data = get_random_joke()

if data:
    print(f"\n{data['value']}")

    show_raw = input("\nShow raw JSON? (y/n): ").lower().strip() == 'y'
    if show_raw:
        print("\n" + "="*50)
        print("RAW JSON RESPONSE")
        print("="*50)
        print(json.dumps(data, indent=2))
else:
    print("Failed to retrieve joke.")

## 2. Fetch available joke categories

**Endpoint:** `GET https://api.chucknorris.io/jokes/categories`

Returns a JSON array of category name strings (e.g. `["animal", "career", "dev", ...]`).
We store the result in `categories` so the next section can validate user input against it.

In [ ]:
def get_categories():
    """
    Fetch the list of available joke categories.

    Returns:
        List of category strings, or None if request fails
    """
    url = "https://api.chucknorris.io/jokes/categories"

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching categories: {e}")
        return None

Run this cell to load and display all available categories. **You must run this before
the "joke by category" cell below**, since that cell checks your input against this list.

In [ ]:
categories = get_categories()

if categories:
    print("Available categories:\n")
    for cat in categories:
        print(f"  - {cat}")
else:
    print("Failed to retrieve categories.")

## 3. Fetch a random joke by category

**Endpoint:** `GET https://api.chucknorris.io/jokes/random?category={category}`

Same response shape as `/jokes/random`, but filtered to a specific category.
The function takes a category string and passes it as a query parameter.

In [ ]:
def get_joke_by_category(category):
    """
    Fetch a random Chuck Norris joke from a specific category.

    Args:
        category: The joke category to fetch from

    Returns:
        Dictionary containing joke data, or None if request fails
    """
    url = f"https://api.chucknorris.io/jokes/random?category={category}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching joke: {e}")
        return None

Enter a category when prompted. If your input doesn't match one of the categories from
section 2, you'll see an error message and can try again without re-running the cell.

In [ ]:
if not categories:
    print("No categories loaded. Run the categories cell above first.")
else:
    while True:
        choice = input(f"Enter a category ({', '.join(categories)}): ").lower().strip()

        if choice in categories:
            data = get_joke_by_category(choice)
            if data:
                print(f"\n[{choice}] {data['value']}")
            else:
                print("Failed to retrieve joke.")
            break
        else:
            print(f"'{choice}' is not a valid category. Please try again.\n")

## 4. Search jokes by keyword

**Endpoint:** `GET https://api.chucknorris.io/jokes/search?query={text}`

Unlike the other endpoints which return a single joke, this one returns an object with:
- `total` — the number of matching jokes
- `result` — a list of joke objects

The API requires queries to be at least **3 characters** long.

In [ ]:
def search_jokes(query):
    """
    Search for Chuck Norris jokes matching a keyword.

    Args:
        query: Search term (minimum 3 characters)

    Returns:
        Dictionary with 'total' count and 'result' list, or None if request fails
    """
    url = f"https://api.chucknorris.io/jokes/search?query={query}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error searching jokes: {e}")
        return None

Enter a search term (at least 3 characters). The cell will display how many jokes matched
and print up to the first 5 results. If your query is too short, you'll be prompted to try again.

In [ ]:
while True:
    query = input("Search for jokes (min 3 characters): ").strip()

    if len(query) >= 3:
        break
    print(f"'{query}' is too short — need at least 3 characters.\n")

data = search_jokes(query)

if data:
    total = data['total']
    results = data['result']

    print(f"\nFound {total} joke(s) matching '{query}':\n")

    for i, joke in enumerate(results[:5], 1):
        print(f"  {i}. {joke['value']}\n")

    if total > 5:
        print(f"  ... and {total - 5} more.")
else:
    print("Failed to search jokes.")